In [1]:
import pandas as pd
import numpy as np
import sklearn
import torch.nn as nn
import sklearn.model_selection
from sklearn import linear_model
from sklearn.preprocessing import MinMaxScaler
import pickle
from sklearn.metrics import accuracy_score,f1_score
pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv("/Users/benjamincheng/Downloads/processed_2019_2024_4.csv", index_col=0)
# use list comphrension to call games from 3/18
january_march_games = df[df['date_next'].str.contains('2024-03') | df['date_next'].str.contains('2024-02')|df['date_next'].str.contains('2024-01')]
january_march_games = january_march_games.reset_index(drop=True)

# Predictors chosen using sequential feature selector trained on ridge classifier
save_path = '/Users/benjamincheng/Documents/GitHub/Sports-Betting/ml_notebooks/factors/'
file_name = 'predictors_ridge_classifier_70_predictors_65.7%_2019_2024.txt'
file_path = save_path + file_name
with open(file_path, 'r') as f:
    predictors = f.read()
    
# model
save_path = '/Users/benjamincheng/Documents/GitHub/Sports-Betting/ml_notebooks/weights/'
file_name = 'ridge_classifier_70_predictors_65.7%_2019_2024.pkl'
file_path = save_path + file_name
with open(file_path, 'rb') as f:
    model = pickle.load(f)

predictors = predictors.split(',')[:-1]
X = january_march_games[predictors]
predictions = model.predict(X)
result = pd.Series(predictions, index = X.index)
final = pd.concat([january_march_games['Teams_x'], result], axis=1)
final.columns = ['Teams', 'Predictions']
final['Actual'] = january_march_games['target_x']

accuracy = accuracy_score(final['Actual'], final['Predictions'])
accuracy

0.6596330275229357

In [10]:
df = pd.read_csv("/Users/benjamincheng/Downloads/cleaned_2019_2024_71.8%_odds.csv", index_col=0)
# use list comphrension to call games from 3/18
january_march_games = df[df['date_next'].str.contains('2024-03') | df['date_next'].str.contains('2024-02')|df['date_next'].str.contains('2024-01')]
january_march_games = january_march_games.reset_index(drop=True)

# Predictors chosen using sequential feature selector trained on ridge classifier
save_path = '/Users/benjamincheng/Documents/GitHub/Sports-Betting/ml_notebooks/factors/'
file_name = 'predictors_ridge_classifier_70_predictors_71.8%_2019_2024.txt'
file_path = save_path + file_name
with open(file_path, 'r') as f:
    predictors = f.read()
    
# model
save_path = '/Users/benjamincheng/Documents/GitHub/Sports-Betting/ml_notebooks/weights/'
file_name = 'ridge_classifier_70_predictors_71.8%_2019_2024.pkl'
file_path = save_path + file_name
with open(file_path, 'rb') as f:
    model = pickle.load(f)

predictors = predictors.split(',')[:-1]
X = january_march_games[predictors]
predictions = model.predict(X)
result = pd.Series(predictions, index = X.index)
final = pd.concat([january_march_games['Teams_x'], result], axis=1)
final.columns = ['Teams', 'Predictions']
final['Actual'] = january_march_games['target_x']

accuracy = accuracy_score(final['Actual'], final['Predictions'])
accuracy

0.7042606516290727

In [11]:
decision_scores = model.decision_function(X)
probabilities = 1 / (1 + np.exp(-decision_scores))

result = pd.DataFrame({
    'Predictions': predictions,
    'Probabilities': probabilities,  # Assuming you want probabilities for the positive class
    'Actual': january_march_games['target_x']
}, index=X.index)

# Concatenate the predictions and probabilities with the original data
final = pd.concat([january_march_games['date_next'],january_march_games['Teams_x'], result], axis=1)
final.columns = ['Date','Teams', 'Predictions', 'Probabilities', 'Actual']

loss = final[final['Predictions']==0]
loss['Probabilities'] = 1 - loss['Probabilities']
win = final[final['Predictions']==1]

complete = pd.concat([win,loss])
complete = complete.reset_index(drop=True)

/var/folders/99/zv3w4flx1598dxlknc099kfr0000gn/T/ipykernel_31359/3215053033.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loss['Probabilities'] = 1 - loss['Probabilities']


In [12]:
complete

,Date,Teams,Predictions,Probabilities,Actual
0,2024-01-09,LAL,1,0.527675,1
1,2024-01-01,LAC,1,0.568208,1
2,2024-01-01,HOU,1,0.587850,1
3,2024-01-01,DEN,1,0.733490,1
4,2024-01-02,PHI,1,0.643458,1
...,...,...,...,...,...
793,2024-02-29,UTA,0,0.577480,0
794,2024-02-29,ATL,0,0.508675,0
795,2024-02-29,SAS,0,0.656933,1
796,2024-02-28,NOP,0,0.555136,0


In [13]:
accuracy_win = accuracy_score(win['Actual'], win['Predictions'])
accuracy_win

0.7052896725440806

In [14]:
accuracy_loss = accuracy_score(loss['Actual'], loss['Predictions'])
accuracy_loss

0.7032418952618454

In [15]:
save_path = "/Users/benjamincheng/Documents/GitHub/Sports-Betting/risk_management/"
file_name = "2024_predictions_win_only.csv"
file_path = save_path + file_name
win.to_csv(file_path)